### Keyword-Only Arguments

As we saw in the lecture we can specify parameters in a function that **must** be passed as named arguments - these are called **keyword-only** arguments.

To do so, Python must know where our positional arguments end - we can do that either by using a `*args` parameter, or just a single `*`.

In [1]:
def func(a, b, *args, c):
    print(a)
    print(b)
    print(args)
    print(c)

In [2]:
func(10, 20, 30, 40, c=50)

10
20
(30, 40)
50


And in fact, we **have** to pass `c` as a named argument:

In [3]:
func(10, 20, 30, 40)

TypeError: func() missing 1 required keyword-only argument: 'c'

Doing that allows extra positional parameters to be passed in though. If we don't want that, we still have to tell Python where the positional arguments end, and we do this by using a single `*`.

In [4]:
def func(a, b, *, c):
    print(a)
    print(b)
    print(c)

In [5]:
func(10, 20, c=30)

10
20
30


And trying to pass additional positional arguments is now allowed:

In [6]:
func(10,  20, 30, c=40)

TypeError: func() takes 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

Of course, we can still pass the positional arguments as named arguments - the only thing here is that `c` **must** be passed as a named argument - up to use how we want to pass the positional arguments:

In [4]:
func(c=30, a=10, b=20)

10
20
()
30


We can specify default values for keyword-only arguments as well, and unlike positional arguments, once we have a default value for one keyword-only argument, the subsequent ones do not have to be defaulted too.

In [5]:
def func(a, b=2, c=3, *, d=10, e, f=30):
    print(a, b, c, d, e, f)

In [6]:
func(1, e=20)

1 2 3 10 20 30


In [7]:
func(e=20, a=1)

1 2 3 10 20 30


In [8]:
func(1, c=3.5, d=100, e=200)

1 2 3.5 100 200 30


As you can see, this system is **very** flexible!

Let's go back to a previous example, and see where we might want to use keyword-only arguments.

In [9]:
def process_data(data, item_sep=',', line_sep='\n'):
    row_strings = [item_sep.join([str(element) for element in row])
                  for row in data]
    return line_sep.join(row_strings)

In [10]:
data = [
    [10, 20, 30],
    [100, 200, 300],
    [1000, 2000, 3000]
]

You'll notice that `item_sep` and `line_sep` are positional arguments, so we can call the function this way:

In [11]:
print(process_data(data, ':', '\n\n'))

10:20:30

100:200:300

1000:2000:3000


Now, it can be confusing as to which positional is used for the item separator, and which one is used for the line separator.

I **choose** to call the function this way:

In [12]:
print(process_data(data, item_sep=':', line_sep='\n\n'))

10:20:30

100:200:300

1000:2000:3000


But ideally, as the developer of this function, I may want to **force** callers of this function to use these named arguments - simply for safety.

So, I would write my function this way:

In [13]:
def process_data(data, *, item_sep=',', line_sep='\n'):
    row_strings = [item_sep.join([str(element) for element in row])
                  for row in data]
    return line_sep.join(row_strings)

Now I can call the function this way:

In [14]:
print(process_data(data, item_sep=':', line_sep='\n\n'))

10:20:30

100:200:300

1000:2000:3000


but I can no longer call it this way:

In [15]:
print(process_data(data, ':', '\n\n'))

TypeError: process_data() takes 1 positional argument but 3 were given

Consider this function, that needs a `latitude` and `longitude` passed to it:

In [16]:
def coords_to_json(longitude, latitude):
    return f'{{"longitude": {longitude}, "latitude": {latitude}}}'

Notice how I used `{{` and `}}` to actually print a `{` and `}` respectively - this allows us to do this without Python getting confused as to whether we are interpolating or just writing a literal brace - this is referred to as "escaping" the character.

Now I can call the function this way:

In [17]:
coords_to_json(10, 20)

'{"longitude": 10, "latitude": 20}'

But, I get easily confused - is the the first argument the latitude or the longitude?

Since I can never remember, I'm always going to pass my arguments as named arguments - much safer for someone like me!

In [18]:
coords_to_json(latitude=20, longitude=10)

'{"longitude": 10, "latitude": 20}'

And in fact, I'm going to write my function so that anyone calling it will be forced to do the same!

In [19]:
def coords_to_json(*, longitude, latitude):
    return f"{{'longitude': {longitude}, 'latitude': {latitude}}}"

So this will work:

In [20]:
coords_to_json(latitude=20, longitude=10)

"{'longitude': 10, 'latitude': 20}"

and this will not:

In [21]:
coords_to_json(10, 20)

TypeError: coords_to_json() takes 0 positional arguments but 2 were given

We also have a mechanism to scoop up extra named arguments into a dictionary that becomes available in the function:

In [22]:
def func(a, b, *args, c, d, **kwargs):
    print(a)
    print(b)
    print(args)
    print(c)
    print(d)
    print(kwargs)

In [23]:
func(10, 20, c=1, d=2, x=100, y=100)

10
20
()
1
2
{'x': 100, 'y': 100}


Even if we specify `a` and `b` as named arguments, they will be correctly assigned to `a` and `b` in our function - not added to the `kwargs` dictionary:

In [24]:
func(c=1, d=2, x=100, y=200, a=10, b=20)

10
20
()
1
2
{'x': 100, 'y': 200}


So we can handle these extra named arguments in a dictionary inside our function. The argument names are the keys, and the argument values are the corresponding dictionary values.

**But, be very careful how you use this feature.**

I often see code written this way:

In [25]:
def func(**kwargs):
    # kwargs should contain 'a' and 'b'
    return kwargs['a'] + kwargs['b']

In [26]:
func(a=1, b=2)

3

This is a silly example, and no one would write it that way, but once you start getting a lot of parameters, and you control how the function is called, it might be tempting to do this:

In [27]:
def func(**kwargs):
    # expect data1, data2, arg1, arg2, arg3, arg4 in kwargs and do something with them
    pass

instead of writing this:

In [28]:
def func(data1, data2, arg1, arg2, arg3, arg4):
    pass

Using `**` is a lazy approach and is not safe - plus someone looking at your function has no idea that you are expecting arguments named `data`, `data2`, `arg1`, etc.

`**` arguments should only be processed in a way that does not require certain keys to be present in the dictionary.

For example, suppose we want to create a string that contains certain mandatory pieces of data, and appends to it anything extra that has been passed in as named arguments:

In [29]:
def to_json(arg1, *, kw1, **extras):
    formatted_extras = ','.join([f'"{key}": {value}' for key, value in extras.items()])
    result = f'{{"arg1": {arg1}, "kw1": {kw1}, "extras": {{{formatted_extras}}}}}'
    return result

In [30]:
print(to_json(10, kw1=20, a=1, b=2, c=3))

{"arg1": 10, "kw1": 20, "extras": {"a": 1,"b": 2,"c": 3}}


We'll see more examples of when to use `**` arguments throughout this course!